### Implementing a Binomial Heap

In [7]:
class BinomialHeapNode:
    def __init__(self, key):
        self.key = key
        self.degree = 0
        self.child = None
        self.sibling = None
        self.parent = None
    
    def print_node(self, level=0):
        """Prints the current node and its children."""
        indent = " " * level * 4  
        print(f"{indent}{self.key} (Degree: {self.degree})")
        
        if self.child:
            self.child.print_node(level + 1)
        if self.sibling:
            self.sibling.print_node(level)  


def find_node_by_key(node, key):
    if node is None:
        return None
    if node.key == key:
        return node
    res = find_node_by_key(node.child, key)
    if res:
        return res
    return find_node_by_key(node.sibling, key)


### Performing Operations

In [8]:
class BinomialHeap:
    def __init__(self):
        self.head = None

    def merge_trees(self, b1: BinomialHeapNode, b2: BinomialHeapNode) -> BinomialHeapNode:
        if b1.key > b2.key:
            b1, b2 = b2, b1  
        b2.parent = b1
        b2.sibling = b1.child
        b1.child = b2
        b1.degree += 1
        return b1

    def union(self, other_heap):
        """Merges two binomial heaps."""
        self.head = self.merge_heaps(self.head, other_heap.head)
        if self.head is None:
            return None
        prev = None
        curr = self.head
        next = curr.sibling
        while next is not None:
            if curr.degree != next.degree or (next.sibling and next.sibling.degree == curr.degree):
                prev = curr
                curr = next
            else:
                if curr.key <= next.key:
                    curr.sibling = next.sibling
                    self.merge_trees(curr, next)
                else:
                    if prev is None:
                        self.head = next
                    else:
                        prev.sibling = next
                    self.merge_trees(next, curr)
                    curr = next
            next = curr.sibling

    def merge_heaps(self, h1, h2):
        """Merges two root lists of binomial heaps in increasing order of degree."""
        if h1 is None:
            return h2
        if h2 is None:
            return h1
        if h1.degree <= h2.degree:
            head = h1
            h1 = h1.sibling
        else:
            head = h2
            h2 = h2.sibling
        tail = head
        while h1 and h2:
            if h1.degree <= h2.degree:
                tail.sibling = h1
                h1 = h1.sibling
            else:
                tail.sibling = h2
                h2 = h2.sibling
            tail = tail.sibling

        tail.sibling = h1 if h1 else h2
        return head

    def insert(self, key):
        """Inserts a key into the binomial heap."""
        new_node = BinomialHeapNode(key)
        temp_heap = BinomialHeap()
        temp_heap.head = new_node
        self.union(temp_heap)

    def get_min(self):
        """Returns the minimum element in the heap."""
        if self.head is None:
            return None

        y = None 
        x = self.head
        min_val = float('inf')

        while x is not None:
            if x.key < min_val:
                min_val = x.key
                y = x
            x = x.sibling

        return y

    def extract_min(self):
        """Extracts the minimum element from the heap."""
        if self.head is None:
            return None

        min_prev = None
        min_node = self.head
        curr = self.head
        prev = None
        min_val = min_node.key

        while curr is not None:
            if curr.key < min_val:
                min_val = curr.key
                min_node = curr
                min_prev = prev
            prev = curr
            curr = curr.sibling

        if min_prev is None:
            self.head = min_node.sibling
        else:
            min_prev.sibling = min_node.sibling

        child = min_node.child
        temp_heap = BinomialHeap()
        while child is not None:
            next = child.sibling
            child.sibling = temp_heap.head
            temp_heap.head = child
            child = next

        self.union(temp_heap)

        return min_node.key

    def decrease_key(self, node, new_key):
        """Decreases the key of a node in the binomial heap."""
        if new_key > node.key:
            return "New key is greater than the current key"

        node.key = new_key
        curr = node
        parent = curr.parent

        while parent is not None and curr.key < parent.key:
            curr.key, parent.key = parent.key, curr.key  # Swap keys
            curr = parent
            parent = curr.parent

    def delete(self, node):
        """Deletes a node from the binomial heap."""
        self.decrease_key(node, float('-inf'))
        self.extract_min()

    def find_node_by_key(node, key):
        """Recursively searches for a node with a given key in the binomial heap."""
        if node is None:
            return None
        
        if node.key == key:
            return node

        found_node = self.find_node_by_key(node.child, key)
        if found_node:
            return found_node

        return self.find_node_by_key(node.sibling, key)
    def print_heap(self):
        """Prints the entire binomial heap."""
        if self.head is None:
            print("Heap is empty.")
            return
        print("Binomial Heap:")
        node = self.head
        while node:
            print(f"Tree with root {node.key}:")
            node.print_node()
            node = node.sibling 

### Testing the heap

In [9]:
# Creating the heap
heap = BinomialHeap()

# Insert elements into the heap
n = int(input("Enter the number of elements you wish to add: "))
for _ in range(n):
    x = int(input("Enter the value of the element: "))
    heap.insert(x)
    heap.print_heap()



# Get the minimum value from the heap
min_node = heap.get_min()
print(f"Minimum value in the heap: {min_node.key}")

# Extract the minimum value from the heap
min_val = heap.extract_min()
print(f"Extracted minimum value: {min_val}")

# Get the minimum value again after extraction
min_node = heap.get_min()
print(f"New minimum value after extraction: {min_node.key}")
x = int(input("Enter the node to decrease its Key: "))
y = int(input("Enter the new value: "))

node_to_decrease = find_node_by_key(heap.head, x)  # Find the node with key x
if node_to_decrease:
    heap.decrease_key(node_to_decrease, y)
    print("Decreased key of node")
else:
    print("Node with key not found")

# Get the new minimum value after decrease key operation
min_node = heap.get_min()
print(f"Minimum value after decreasing key: {min_node.key}")

# Extract the new minimum value (which should now be 1)
min_val = heap.extract_min()
print(f"Extracted new minimum value: {min_val}")

# Get the minimum value again
min_node = heap.get_min()
print(f"New minimum value after extracting 1: {min_node.key}")

# Delete the minimum node
min_node = heap.get_min()
print(f"Deleting node with key: {min_node.key}")
heap.delete(min_node)

# Get the minimum value after deletion
min_node = heap.get_min()
if min_node:
    print(f"New minimum value after deletion: {min_node.key}")
else:
    print("Heap is empty after deletion.")


Binomial Heap:
Tree with root 20:
20 (Degree: 0)
Binomial Heap:
Tree with root 10:
10 (Degree: 1)
    20 (Degree: 0)
Binomial Heap:
Tree with root 15:
15 (Degree: 0)
10 (Degree: 1)
    20 (Degree: 0)
Tree with root 10:
10 (Degree: 1)
    20 (Degree: 0)
Binomial Heap:
Tree with root 4:
4 (Degree: 2)
    10 (Degree: 1)
        20 (Degree: 0)
    15 (Degree: 0)
Binomial Heap:
Tree with root 2:
2 (Degree: 0)
4 (Degree: 2)
    10 (Degree: 1)
        20 (Degree: 0)
    15 (Degree: 0)
Tree with root 4:
4 (Degree: 2)
    10 (Degree: 1)
        20 (Degree: 0)
    15 (Degree: 0)
Minimum value in the heap: 2
Extracted minimum value: 2
New minimum value after extraction: 4
Decreased key of node
Minimum value after decreasing key: 4
Extracted new minimum value: 4
New minimum value after extracting 1: 6
Deleting node with key: 6
New minimum value after deletion: 10


### Time complexity Analysis:
- Insertion: 

    Best : O(1) -> Adding into an empty heap 
    
    Worst : O(log(n)) -> May need to merge the trees
- Finding Min: 
    
    O(log(n)) -> Coz searching through root list
- Extract Min:

    Best : O(log(n))
    
    Worst : O(log(n))
- Union: 

    Best : O(1) -> Both heaps have no trees with same degree

    Worst: O(log(n)) -> Because Reorganisation is needed
- Decrease Key: 

    Best: O(1) -> No rearranging of nodes required

    Worst : O(log(n)) -> As node may be moved up
- Delete: 

    Best: O(log(n)) -> Decrease Key followes by extract minimum

    Worst : O(log(n))